<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Task-setting" data-toc-modified-id="Task-setting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Task setting</a></span><ul class="toc-item"><li><span><a href="#Validation-criteria" data-toc-modified-id="Validation-criteria-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Validation criteria</a></span></li></ul></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Split" data-toc-modified-id="Split-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Split</a></span></li><li><span><a href="#Content-recomendations" data-toc-modified-id="Content-recomendations-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Content recomendations</a></span><ul class="toc-item"><li><span><a href="#Index" data-toc-modified-id="Index-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Index</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Checks-recomendations-Word2vec" data-toc-modified-id="Checks-recomendations-Word2vec-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Checks recomendations Word2vec</a></span><ul class="toc-item"><li><span><a href="#Model-word2vec" data-toc-modified-id="Model-word2vec-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Model word2vec</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Recomendations-sys-2-staged" data-toc-modified-id="Recomendations-sys-2-staged-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Recomendations sys 2 staged</a></span><ul class="toc-item"><li><span><a href="#Generation-of-candidates" data-toc-modified-id="Generation-of-candidates-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Generation of candidates</a></span></li><li><span><a href="#Ranking" data-toc-modified-id="Ranking-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Ranking</a></span></li></ul></li></ul></div>

**Рекомендательная система для  аптеки **

## Task setting

Разработать рекомендательную систему для аптек. Представьте, что вы приходите на кассу, а у кассира во время формирования чека всплывает подсказка, что можно вам ещё порекомендовать.

### Validation criteria

    Построен валидационный датасет. Валидация сделана с учётом времени либо обосновано, почему валидация в данном случае бессмысленна	2
    Построены контентные рекомендации (по названиям товаров в корзине)	4
    Построены рекомендации с Word2Vec по чековым данным (обучены эмбеддинги товаров по чековым последовательностям)	4
    Построена двухстадийная рекомендательная система (опционально, с любым ранжированием)	2

## Imports 

In [2]:
# Data works 
import pandas as pd 
import numpy as np 
import pickle

#Text works 
import string
import annoy 
import pymorphy2 
from stop_words import get_stop_words


# Text vectorizing
from gensim.models import Word2Vec
from gensim.models import FastText 

# Splitting data 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

# 
from lightfm import LightFM 
from lightfm.evaluation import precision_at_k 

C:\Users\SElen\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


## Data 

Перечень товаров и  id

In [3]:
with open('Product_dict.pkl', 'rb') as f: 
    product_dict = pickle.load(f)
    

In [4]:
product_dict

{'168308': '(197312) Пакет-майка 25см х 45см 906',
 '134832': '(62448) Перекись водорода р-р наружн. 3% фл.полимерн. 100мл 404',
 '101384': '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404',
 '168570': '(197309) Пакет 28см х 50см 906',
 '146960': '(111023) Пакет "Аптека Озерки" 28 х 35см 906',
 '128700': '(51806) Ранитидин тб 150мг уп N20 595',
 '52539': '(19416) Платки носовые "Зева Кидс" №10 721',
 '94270': '(66684) Пакет майка "Благодарим за покупку" (28+14) х 43см №1 -1',
 '77426': '(5605) Иглы Микро-Файн д/шприц-ручки 30G 0,3*8мм N100 759',
 '92475': '(114214) Дорзопт капли глазные 2% 5мл. фл.-кап. N3 575',
 '158643': '(181230) PL Шприц одноразовый 3-комп. 3мл №1 762',
 '158600': '(181232) PL Шприц одноразовый 3-комп. 5мл №1 762',
 '106230': '(43344) Аллапинин тб 25мг N30 738',
 '110629': '(57733) Корвалол капли д/приема внутрь 25мл 738',
 '47251': '(121286) PL Контейнер д/биопроб стер. 60мл с крыш и ложкой в инд.уп 711',
 '39811': '(116543) PL Салфетка антисептическая спи

Чековые данные 

In [5]:
data = pd.read_csv('чековые данные.csv')

C:\Users\SElen\AppData\Local\Temp\ipykernel_13656\3965517632.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('чековые данные.csv')


## Preprocessing

In [6]:
checks = data.dropna().copy()
checks=checks.drop_duplicates()
checks.head()

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00"


In [7]:
checks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19703850 entries, 0 to 19999999
Data columns (total 9 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   sale_date_date           object 
 1   contact_id               object 
 2   shop_id                  float64
 3   product_id               float64
 4   name                     object 
 5   product_sub_category_id  float64
 6   product_category_id      float64
 7   brand_id                 float64
 8   quantity                 object 
dtypes: float64(5), object(4)
memory usage: 1.5+ GB


In [8]:
# Приведем даты в надлежащий формат и переименуем, удалив ошибку
checks = checks.rename(columns={'sale_date_date': 'date'})
checks = checks[checks['date'].str.startswith("20")]
checks['date'] = pd.to_datetime(checks['date'])


Для построения word to vector рекомендаций на основе эмбеддингов в чековых последовательностях, нам нужно связать информацию о чеке, магазине и дате, чтобы потом сгруппировать и получить список товаров в каждом чеке. 


In [9]:
checks['purchase'] = checks['date'].apply(str) + "_" + checks[
    'contact_id'].apply(str) + "_" + checks['shop_id'].apply(str)

In [10]:
# quantity определенно должен быть численным признаком. Заменю запятые и тип 
checks['quantity'] = checks['quantity'].apply(lambda x: x.replace(',', '.')).astype(float)

In [11]:
checks['product_id'] = checks['product_id'].apply(int).apply(str)

In [12]:
float_cols = checks.select_dtypes(include=['float64']).columns[:-1]
float_cols

Index(['shop_id', 'product_sub_category_id', 'product_category_id',
       'brand_id'],
      dtype='object')

Вообще говоря, конечно эти признаки не числа, а по смыслу - категории. 

In [13]:
for col in float_cols:
    checks[col] = checks[col].apply(int).apply(str)

In [14]:
checks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19703850 entries, 0 to 19999999
Data columns (total 10 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   date                     datetime64[ns]
 1   contact_id               object        
 2   shop_id                  object        
 3   product_id               object        
 4   name                     object        
 5   product_sub_category_id  object        
 6   product_category_id      object        
 7   brand_id                 object        
 8   quantity                 float64       
 9   purchase                 object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 1.6+ GB


## Split 

Данные упорядочены по времени, используем это для построения валидационного сета, поскольку у нас нет пользовательских сессий.

In [15]:
train, test = train_test_split(checks, test_size=0.3)

In [16]:
print(train.shape)
print(test.shape)

(13792695, 10)
(5911155, 10)


## Content recomendations

In [17]:
# Очистка от пунктуации 
punctuation = set(string.punctuation)

# стоп слова
stop_words = set(get_stop_words('ru'))

# лемматизация 
morpher = pymorphy2.MorphAnalyzer()

In [18]:
def preprocess_text(line):
    '''Prod_name cleaning from punctuation, register unification, 
    returning normal form, excluding stop words'''
    item = "".join(i for i in str(line).strip()
                   if i not in punctuation).split()
    item = [morpher.parse(i.lower())[0].normal_form for i in item]
    item = [i for i in item if i not in stop_words and i != ""]
    return item

In [19]:
sentences = [preprocess_text(product_dict[k]) for k in product_dict.keys()]
    

In [20]:
sentences

[['197312', 'пакетмайк', '25см', 'х', '45см', '906'],
 ['62448',
  'перекись',
  'водород',
  'рр',
  'наружн',
  '3',
  'флполимерна',
  '100мл',
  '404'],
 ['72183',
  'салициловый',
  'кислота',
  'рр',
  'спирт',
  '2',
  'фл',
  '40мл',
  'n1',
  '404'],
 ['197309', 'пакет', '28см', 'х', '50см', '906'],
 ['111023', 'пакет', 'аптека', 'озерко', '28', 'х', '35см', '906'],
 ['51806', 'ранитидина', 'тб', '150мг', 'уп', 'n20', '595'],
 ['19416', 'платок', 'носовой', 'зев', 'кидс', '№10', '721'],
 ['66684',
  'пакет',
  'майк',
  'благодарить',
  'покупка',
  '2814',
  'х',
  '43см',
  '№1',
  '1'],
 ['5605', 'игла', 'микрофайн', 'дшприцручка', '30g', '038мма', 'n100', '759'],
 ['114214', 'дорзопт', 'капля', 'глазной', '2', '5мл', 'флкап', 'n3', '575'],
 ['181230', 'pl', 'шприц', 'одноразовый', '3комп', '3мл', '№1', '762'],
 ['181232', 'pl', 'шприц', 'одноразовый', '3комп', '5мл', '№1', '762'],
 ['43344', 'аллапинин', 'тб', '25мг', 'n30', '738'],
 ['57733', 'корвалола', 'капля', 'дприём

In [21]:
model_fast_text = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

In [22]:
model_fast_text.save('model_fast_text_rec_sys_pharma')

### Index 

In [23]:
fast_text_index = annoy.AnnoyIndex(20, 'angular')

In [24]:
index_map = {} # пары  счетчик: prod_id
#reverse_index_map = {} # пары prod_id: счетчик 
counter = 0

# для каждого описания товара в перечне 
for prod_id in product_dict.keys(): 
    # включаем счетчик для слов в описании
    n_ft = 0
    # пишем в индексе номер описания и описание
    index_map[counter] = prod_id
    
    #reverse_index_map[counter] = name
    # открываем нулевой вектор рамера 20
    vector_ft = np.zeros(20)
    
    # каждое 'слово' в предобработанном описании
    for word in preprocess_text(product_dict[prod_id]):
        # если слово записано моделью fast_text
        if word in model_fast_text.wv:
            # к нулевому вектору описания прибавляем вектор слова
            vector_ft += model_fast_text.wv[word]
            # увеличиваем счетчик слов в описаниии
            n_ft += 1
    # для описаний более одного слова проводим простое усреднение вектора на количество слов в нем        
    if n_ft > 0: 
        vector_ft = vector_ft / n_ft
    # добавляем в индекс полученный вектор описания товара
    fast_text_index.add_item(counter, vector_ft)
    # счетчик на новую строку 
    counter += 1
    

In [25]:
# строим индекс
fast_text_index.build(10)

True

In [26]:
for item in ["52539", "110342"]:
    print(product_dict[item])

(19416) Платки носовые "Зева Кидс" №10 721
(64664) Фосфалюгель гель 16г N20 595


In [27]:
# строим обьратный индекс для быстрого получения отметки счетчика на prod_id 
reverse_index_id_cnt = {v: k for k, v in index_map.items()}
reverse_index_id_cnt

{'168308': 0,
 '134832': 1,
 '101384': 2,
 '168570': 3,
 '146960': 4,
 '128700': 5,
 '52539': 6,
 '94270': 7,
 '77426': 8,
 '92475': 9,
 '158643': 10,
 '158600': 11,
 '106230': 12,
 '110629': 13,
 '47251': 14,
 '39811': 15,
 '143709': 16,
 '25965': 17,
 '158645': 18,
 '79879': 19,
 '100667': 20,
 '87332': 21,
 '21249': 22,
 '158659': 23,
 '76206': 24,
 '114366': 25,
 '34531': 26,
 '27063': 27,
 '20236': 28,
 '99525': 29,
 '116091': 30,
 '97117': 31,
 '160521': 32,
 '94899': 33,
 '67922': 34,
 '98991': 35,
 '90247': 36,
 '28232': 37,
 '60404': 38,
 '19578': 39,
 '124839': 40,
 '42409': 41,
 '77808': 42,
 '60228': 43,
 '110342': 44,
 '124535': 45,
 '79735': 46,
 '68982': 47,
 '135018': 48,
 '141455': 49,
 '148629': 50,
 '55357': 51,
 '135427': 52,
 '19510': 53,
 '66818': 54,
 '141996': 55,
 '161562': 56,
 '155688': 57,
 '155686': 58,
 '162991': 59,
 '87663': 60,
 '145608': 61,
 '108808': 62,
 '127911': 63,
 '141757': 64,
 '153569': 65,
 '115144': 66,
 '32087': 67,
 '71618': 68,
 '114395'

In [28]:
def recommend(ids_list):
    """Getting 3 approx neighbours to each item_id given, 
    based on content"""
    iid_c = len(ids_list)
    iid_vector = np.zeros(20)
    for iid in ids_list:
        if iid not in reverse_index_id_cnt:
            iid_c -= 1
            continue
        iid_vector += np.array(
        fast_text_index.get_item_vector(reverse_index_id_cnt[iid]))
    
    return [product_dict[index_map[idx]]
         for idx in fast_text_index.get_nns_by_vector(iid_vector / iid_c, 10)]

### Test


In [29]:
recommend(['99821', '138583', '45321', '134475'])

['(185260) Баета Лонг пор.для п/к.введ.пролонг. 2мг шприц-ручка 0,65мл №4 462',
 '(91468) Гонал Ф 300МЕ шпр.-руч. 22мкг/0,5мл+иглы №8 448',
 '(67759) Маалокс мини суспензия саше 4,3мл №6 595',
 '(96732) Калия йодид Реневал 200 мкг №112 Renewal 458',
 '(90925) Алдара крем для наружного применения 5% пак. 250 мг №12 581',
 '(90605) Цетротид лиофилизат для приготовления раствора для подкожного введения 250мг флакон с раство 459',
 '(119825) Динамико Форвард пленки дисперг.в полости рта 100мг саше №1 683',
 '(96731) Калия йодид Реневал 100 мкг №112 Renewal 458',
 '(119826) Динамико Форвард пленки дисперг.в полости рта 50мг саше №1 683',
 '(271266) PL Аскорбиновая кислота со вкусом малины 500 мг порошок №1 429']

## Checks recomendations Word2vec 

Сгруппируем данные по чекам с дополнительными признаками - по последовательностям.

In [30]:
# группируем данные по покупкам  
grouped = train.groupby('purchase')
sentences = []

In [31]:
for group in grouped.groups:
    products = grouped.get_group(group)['product_id'].values # из каждой покупки забираем id товаров в чеке
    if len(products) < 3: # пропустим слишком короткие чеки - где товаров всего 3 
        continue
    sentences.append(list(products))

In [32]:
sentences[:10]

[['134475', '45321', '99821', '82717'],
 ['144740', '70976', '113241', '44789'],
 ['70772', '108353', '37779', '137539'],
 ['113585', '56622', '19543', '92304'],
 ['21252', '148319', '142938', '77651', '94105'],
 ['118593', '105944', '127362', '137539', '100907', '74433', '78852', '41890'],
 ['102814', '85456', '144743'],
 ['98132', '82793', '55888', '93853', '70496'],
 ['77808', '128672', '106107'],
 ['79507', '71384', '70642']]

### Model word2vec

In [33]:
model_w2v = Word2Vec(sentences, vector_size=10)

In [34]:
model_w2v.save('model_check_list')

In [35]:
def recommend_checks(items_list):
    """returns a list of item names which have figuered 
    in checks with the given ones"""
    vector = np.zeros(10)
    counter = len(items_list)
    for iid in items_list: 
        if iid not in model_w2v.wv:
            counter -= 1
            continue
        vector += model_w2v.wv[iid]
    if counter >= 0: 
        vector = vector / counter 
    recommendations = model_w2v.wv.most_similar(vector, topn=10) 
    return [product_dict[item[0]] for item in recommendations]
        
            
    

### Test 


In [36]:
recommend_checks(['99821', '138583', '45321', '134475'])

['(115138) Монтелукаст таб.п.п.о.10мг №30 550',
 '(101136) Серетид аэроз. д/ингал. доз. 25 мкг+125 мкг/доза 120 доз. 550',
 '(114196) Пульмикорт сусп д/инг 0,5мг/мл контейнер 2мл N20 395',
 '(271304) Лазолван р-р для приема внутрь и д/ингал. 7,5мг/мл фл. 100 мл 700',
 '(13778) Беродуал р-р д/ингал. фл.-кап. темн. стекл. 20 мл. 550',
 '(121015) Момат Рино спрей наз.доз.50мкг/доза 120доз фл.с доз. 459',
 '(112634) Ингалятор компрессорный CN-233 584',
 '(110018) Дезринит спрей наз.доз.50мкг/доза фл.18г 395',
 '(12866) Лазолван р-р для приема внутрь и д/ингал. 7,5мг/мл фл. 100 мл 700',
 '(120166) Ингалятор компрессорный LD-212С компактный маски д/взрослых и детей белый 584']

## Recomendations sys 2 staged

### Generation of candidates

Для дальнейшей работы возьмем рекомендации, полученные при контекстном подходе, и рекомендации, полученные при работе с чековыми последовательностями  на первом этапе и проведем некоторое ранжирование полученных кандидатов на втором этапе. 

In [37]:
def candidate_generator(items_list):
    """Returns a set of candidates for later ranking based as sum of results of context 
    recomendations and lists of items in checks """
    for item in items_list:
        if item is not None: 
            content_recommended = list(recommend(items_list))
            checks_recommended = list(recommend_checks(items_list)) 
    return set([i for i in content_recommended + checks_recommended])

In [38]:
candidate_generator(['99821', '138583', '45321', '134475'])

{'(101136) Серетид аэроз. д/ингал. доз. 25 мкг+125 мкг/доза 120 доз. 550',
 '(110018) Дезринит спрей наз.доз.50мкг/доза фл.18г 395',
 '(112634) Ингалятор компрессорный CN-233 584',
 '(114196) Пульмикорт сусп д/инг 0,5мг/мл контейнер 2мл N20 395',
 '(115138) Монтелукаст таб.п.п.о.10мг №30 550',
 '(119825) Динамико Форвард пленки дисперг.в полости рта 100мг саше №1 683',
 '(119826) Динамико Форвард пленки дисперг.в полости рта 50мг саше №1 683',
 '(120166) Ингалятор компрессорный LD-212С компактный маски д/взрослых и детей белый 584',
 '(121015) Момат Рино спрей наз.доз.50мкг/доза 120доз фл.с доз. 459',
 '(12866) Лазолван р-р для приема внутрь и д/ингал. 7,5мг/мл фл. 100 мл 700',
 '(13778) Беродуал р-р д/ингал. фл.-кап. темн. стекл. 20 мл. 550',
 '(185260) Баета Лонг пор.для п/к.введ.пролонг. 2мг шприц-ручка 0,65мл №4 462',
 '(271266) PL Аскорбиновая кислота со вкусом малины 500 мг порошок №1 429',
 '(271304) Лазолван р-р для приема внутрь и д/ингал. 7,5мг/мл фл. 100 мл 700',
 '(67759) М

### Ranking

Итак, список кандидатов сформирован и теперь нужно некоторым образом их отранжировать. 
Логика ранжирования может быть гибкой и изменяться в зависимости от текущих требований, адаптируясь под истекающие сроки реализации, популярность товара в текущий период, цену, дополняющую чек до ровной суммы и другое. 
С учетом имеющихся в распоряжении данных ранжирование будет построено на основе частоты покупок. 

In [53]:
rank_base = checks.product_id.value_counts().to_dict() 

In [56]:
rank=pd.DataFrame.from_dict(rank_base, orient='index')
rank.reset_index(inplace=True)      

In [57]:
rank.columns=['product_id', 'freq']

In [58]:
rank

,product_id,freq
0,129195,147580
1,168308,117367
2,158662,88411
3,110629,85714
4,137539,76555
...,...,...
36107,222212,1
36108,38449,1
36109,235206,1
36110,217096,1


In [59]:
product_dict_df = pd.DataFrame.from_dict(product_dict, orient='index')
product_dict_df.reset_index(inplace=True)

In [64]:
product_dict_df.head()

,product_id,name
0,168308,(197312) Пакет-майка 25см х 45см 906
1,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...
2,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...
3,168570,(197309) Пакет 28см х 50см 906
4,146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906"


In [63]:
product_dict_df.columns=['product_id', 'name']

In [66]:
rank = rank.merge(product_dict_df, on ='product_id', copy=False)    

In [67]:
rank.head()

,product_id,freq,name
0,129195,147580,(48791) Свободная сумма НДС18% (служебный) -1
1,168308,117367,(197312) Пакет-майка 25см х 45см 906
2,158662,88411,(181542) Карта Забота о Здоровье 855
3,110629,85714,(57733) Корвалол капли д/приема внутрь 25мл 738
4,137539,76555,(117825) Панангин таб.п.п.о.№50 738


Теперь немного перепишу функцию рекомендации, добавив упорядочивание по рангу.

In [76]:
def candidate_generator_sort(items_list):
    """Returns a set of candidates for later ranking based as sum of results of context 
    recomendations and lists of items in checks """
    for item in items_list:
        content_recommended = recommend(items_list)
        checks_recommended = recommend_checks(items_list)
    offer = content_recommended + checks_recommended  # получаем список наименований товаров
    offer_ranked = rank[rank['name'].isin(offer)].sort_values(
        by='freq', ascending=False)[['product_id', 'name']]

    return offer_ranked

In [77]:
candidate_generator_sort(['99821', '138583', '45321', '134475'])

,product_id,name
156,149722,(13778) Беродуал р-р д/ингал. фл.-кап. темн. с...
495,82717,(12866) Лазолван р-р для приема внутрь и д/инг...
554,33243,(110018) Дезринит спрей наз.доз.50мкг/доза фл....
2048,152753,(121015) Момат Рино спрей наз.доз.50мкг/доза 1...
2382,90984,(112634) Ингалятор компрессорный CN-233 584
2783,350753,(271304) Лазолван р-р для приема внутрь и д/ин...
4358,59493,(120166) Ингалятор компрессорный LD-212С компа...
4366,72299,(115138) Монтелукаст таб.п.п.о.10мг №30 550
4467,105911,"(67759) Маалокс мини суспензия саше 4,3мл №6 595"
4615,112607,(101136) Серетид аэроз. д/ингал. доз. 25 мкг+1...


<div class="burk">
</div><i class="fa fa-lightbulb-o "></i> Спасибо за внимание к этому проекту :)